# MeshUtility

https://github.com/zishun/MeshUtility

## Functions & Modules
- ```colormap_vertex_color```: assign vertex color to visualize a scalar field defined on mesh.
- ```cut_along_curve```: cut a mesh along a vertex chain.
- ```ff_graph```: face-face graph of mesh.
- ```get_scalar_field_on_resampled_points```: given a scalar field defined on a triangular mesh, get field values on resampled points.
- ```IsoCurve``` module: extract isocurves on a scalar field defined on a manifold triangular mesh.
- ```pygeodesic``` module (C++): geodesic for triangular meshes.
    - exact geodesic by Danil Kirsanov.
    - fast marching, may use different speed on each face.
- ```pyisocurve``` module (C++): almost the same as ```IsoCurve``` above.
- ```pyremesh``` module (C++): incremental isotropic remeshing.
- ```pyshapeop``` module (C++): a partial binding of ShapeOp.
- ```read_obj_lines```: read polyline from a [Wavefront .obj file](https://en.wikipedia.org/wiki/Wavefront_.obj_file#Line_elements).
- ```remove_unreferenced_vertices```: remove unreferenced vertices.
- ```sphere_cvt```: iteratively approximate centroidal Voronoi tessellation (CVT) on the unit sphere (kind of uniform sampling).
- ```split_connected_components```: split connected components.
- ```split_mesh, split_mesh_complete```: split a mesh by inserting new vertices defined on mesh edges.
- ```write_obj_lines```: write polyline as a Wavefront .obj file that can be open with MeshLab.
- ```vv_graph```: vertex-vertex graph of mesh.


## Install

* Local machine: ```pip install meshutility==0.0.2```.
* Google Colab: python3.7 builds ```openmesh``` from source. Here we use ```meshplot``` for visualization. Let's install them seperately.

In [ ]:
!pip install openmesh pythreejs
!pip install git+https://github.com/skoch9/meshplot@v0.3.2

In [ ]:
!pip install meshutility==0.0.2

In [ ]:
# some necessary preparation
import meshutility as mu
import numpy as np
import openmesh as om
import networkx as nx
import meshplot as mp
from IPython.core.display import display, HTML

# # colab may suggest to add the following two lines. DO NOT DO IT!
# from google.colab import output
# output.enable_custom_widget_manager()

def display_viewer(viewer):
    display(HTML(viewer.to_html()))

def mp_plot(*args, **kwargs):
    display_viewer(mp.plot(return_plot=True, *args, **kwargs))

## Sphereical CVT

In [ ]:
verts, faces = mu.sphere_cvt(100, iters=100)
mp_plot(verts, faces)

In [ ]:
mp_plot(verts, c=verts, shading={"point_size": 0.3})

## Iso-curve

In [ ]:
field = verts[:, 1]

isocurve = mu.IsoCurve(verts, faces, field)
pts, on_edges, ratios, isocurve_indices = isocurve.extract(0.5)

In [ ]:
p = mp.plot(verts, faces, c=field, return_plot=True)
edges = []
for line in isocurve_indices:
    edges.extend([[line[i], line[i+1]] for i in range(len(line)-1)])
edges = np.array(edges, 'i')
p.add_edges(pts, edges, shading={"line_color": "red"})
display_viewer(p)

## Remesh

In [ ]:
mesh = om.TriMesh(verts, faces)
remesher = mu.pyremesh.remesher()
remesher.init_mesh(verts, faces)

ev = mesh.ev_indices()
edges = verts[ev[:,0]] - verts[ev[:,1]]
edge_lengths = np.linalg.norm(edges, axis=1)
remesher.remesh(np.median(edge_lengths)*0.6, 15)
verts1, faces1 = remesher.get_mesh()
d = mp.subplot(verts, faces, c=field, s=[1, 2, 0])
field1 = mu.get_scalar_field_on_resampled_points(verts, faces, field, verts1)
mp.subplot(verts1, faces1, c=field1, s=[1, 2, 1], data=d)
display_viewer(d)

## Mesh Splitting/Cutting

In [ ]:
mesh_split, curve_idx = mu.split_mesh(mesh.points(),
                             mesh.fv_indices(),
                             on_edges, ratios)

np.random.seed(5)
d = mp.subplot(verts, faces, c=np.random.rand(*faces.shape), s=[1, 2, 0])
np.random.seed(5)
mp.subplot(mesh_split.points(), mesh_split.fv_indices(), c=np.random.rand(mesh_split.n_faces(), 3), s=[1, 2, 1], data=d)
display_viewer(d)

In [ ]:
curve_idx_ring = curve_idx + [curve_idx[0]]
curve = [curve_idx_ring[v] for v in isocurve_indices[0]]
mesh_cut, curve_idx = mu.cut_along_curve(mesh_split.points(),
                                         mesh_split.fv_indices(),
                                         curve)
parts = mu.split_connected_components(mesh_cut.points(),
                                      mesh_cut.fv_indices())
n = len(parts)
d = mp.subplot(parts[0].points(), parts[0].fv_indices(), s=[1, n, 0])
for i in range(1, n):
    mp.subplot(parts[i].points(), parts[i].fv_indices(), s=[1, n, i], data=d)
display_viewer(d)

## Geodesic Distance Field

In [ ]:
u, v = 0, 20
path_edge, path_ratio = mu.pygeodesic.find_path(verts, faces, u, v)

pts0 = verts[path_edge[:,0]]
pts1 = verts[path_edge[:,1]]
pts = np.multiply(pts0, 1.-path_ratio[:, np.newaxis]) + \
            np.multiply(pts1, path_ratio[:, np.newaxis])
p = mp.plot(verts, faces, return_plot=True)
edges = [[i, i+1] for i in range(pts.shape[0]-1)]
edges = np.array(edges, 'i')
p.add_edges(pts, edges, shading={"line_color": "red"})
display_viewer(p)

In [ ]:
mesh_split, source = mu.split_mesh(verts, faces,
                              path_edge, path_ratio)

# compute geodesic distance field
field = mu.pygeodesic.distance_field(mesh_split.points(),
                                     mesh_split.fv_indices(),
                                     source, 0.05)

mp_plot(mesh_split.points(), mesh_split.fv_indices(), c=field)

## Mesh to Graph

In [ ]:
G = mu.ff_graph(mesh)
assert(mesh.n_faces() == G.number_of_nodes())
print('#components=%d' % (nx.number_connected_components(G)))
nx.draw(G, node_size=40, pos=nx.spring_layout(G, seed=2))

In [ ]:
G = mu.ff_graph(mesh_cut)
assert(mesh_cut.n_faces() == G.number_of_nodes())
print('#components=%d' % (nx.number_connected_components(G)))
nx.draw(G, node_size=40, pos=nx.spring_layout(G, seed=11))